# VacationPy

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
city_data_df = pd.read_csv('../output_data/citydata.csv')
city_data_df.head()
len(city_data_df)                           

561

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Extract latitude and longitude
lat = city_data_df["Lat"].fillna(0)
lng = city_data_df["Lng"].fillna(0)

# Concatinate latitude and longitude and add it to a list
coordinates = list(zip(lat,lng))

figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin':'0 auto 0 auto'
    
}

fig = gmaps.figure(layout=figure_layout)

In [49]:
markers = gmaps.heatmap_layer(coordinates,weights = city_data_df['Humidity'])
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
# Narrow down records that match criteria:
    # Cloudiness = 0, Max Temp  > 20 but < 90 and Wind Speed <= 10
condition = city_data_df[(city_data_df['Cloudiness'] > 0) | ((city_data_df['Max Temp'] <= 20) | (city_data_df['Max Temp'] >= 90)) | (city_data_df['Wind Speed'] > 10)]
reduced_df = city_data_df.drop(condition.index, axis = 0, inplace= False)
reduced_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,Haverfordwest,0,GB,1584337043,80,51.80,-4.97,35.01,3.36
15,Rosetta,0,EG,1584336916,67,31.40,30.42,61.88,4.70
18,Shache,0,CN,1584337046,16,38.42,77.24,54.81,4.07
42,Borod,0,RO,1584337049,100,46.98,22.63,28.40,5.82
54,Guararapes,0,BR,1584337050,89,-21.26,-50.64,72.10,3.89
69,Boddam,0,GB,1584337052,84,57.47,-1.78,33.01,8.05
86,Alghero,0,IT,1584337054,87,40.56,8.32,46.40,5.82
89,Lebu,0,CL,1584337055,96,-37.62,-73.65,52.07,6.04
95,Mar del Plata,0,AR,1584337055,93,-38.00,-57.56,59.00,2.24
121,Lüderitz,0,NaN,1584337059,82,-26.65,15.16,60.80,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map